In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-and-reddit-sentimental-analysis-dataset/Twitter_Data.csv
/kaggle/input/twitter-and-reddit-sentimental-analysis-dataset/Reddit_Data.csv


In [ ]:
pip install torchtext

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
import torchtext
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader

twitter=pd.read_csv('../input/twitter-and-reddit-sentimental-analysis-dataset/Twitter_Data.csv')
#reddit=pd.read_csv('Reddit_Data.csv')
twitter.dropna(axis=0, inplace=True)
train=[]
train=list(twitter['clean_text'])
#for i in reddit['clean_comment']:
#  train.append(str(i))
labels=[]
labels=list(twitter['category'])
#for i in reddit['category']:
#  labels.append(i)

from string import punctuation
punctuation+="“"
punctuation+="”"
# print(punctuation)

# print(punctuation)
print(train[0])
all_text=""
# get rid of punctuation
for i in range(len(train)):
  train[i]=str(train[i]).lower()
  train[i]=''.join([c for c in train[i] if c not in punctuation])
  all_text+=train[i]+" "

words = all_text.split()

from collections import Counter

## Build a dictionary that maps words to integers
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab = ['unk']+vocab
# print(vocab)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}
## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
reviews_ints = []
for review in train:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])

encoded_labels = np.array([i+1 for i in labels])
# print(encoded_labels[:10])

review_lens = Counter([len(x) for x in reviews_ints])

print('Number of reviews before removing outliers: ', len(reviews_ints))

## remove any reviews/labels with zero length from the reviews_ints list.

# get indices of any reviews with length 0
non_zero_idx = [ii for ii, review in enumerate(reviews_ints) if len(review) != 0]

# remove 0-length reviews and their labels
reviews_ints = [reviews_ints[ii] for ii in non_zero_idx]
encoded_labels = np.array([encoded_labels[ii] for ii in non_zero_idx])

print('Number of reviews after removing outliers: ', len(reviews_ints))

def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    
    # getting the correct rows x cols shape
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)

    # for each review, I grab that review and 
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

seq_length = 200
features = pad_features(reviews_ints, seq_length=seq_length)

when modi promised “minimum government maximum governance” expected him begin the difficult job reforming the state why does take years get justice state should and not business and should exit psus and temples
Number of reviews before removing outliers:  162969
Number of reviews after removing outliers:  162968


In [ ]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(130374, 200) 
Validation set: 	(16297, 200) 
Test set: 		(16297, 200)


In [ ]:
np.any(np.isnan(val_y))

False

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

rf = RandomForestClassifier(n_estimators=500)
rf.fit(train_x, train_y)
print ("Accuracy: %s" 
       % accuracy_score(val_y, rf.predict(val_x)))

Accuracy: 0.5397925998650058


In [ ]:
from sklearn.naive_bayes import MultinomialNB
#training the model
mnb=MultinomialNB()
#fitting the svm for bag of words
mnb_bow=mnb.fit(train_x,train_y)
print(mnb_bow)

MultinomialNB()


In [ ]:
#Predicting the model for bag of words
mnb_bow_predict=mnb.predict(test_x)
print(mnb_bow_predict)

[1. 0. 1. ... 1. 1. 0.]


In [ ]:
#Accuracy score for bag of words
mnb_bow_score=accuracy_score(test_y,mnb_bow_predict)
print("mnb_bow_score :",mnb_bow_score)

mnb_bow_score : 0.39927594035712094


In [ ]:
pip install scikit-learn  -U

     |████████████████████████████████| 22.3 MB 151 kB/s eta 0:00:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.23.2
    Uninstalling scikit-learn-0.23.2:
      Successfully uninstalled scikit-learn-0.23.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyldavis 3.3.1 requires numpy>=1.20.0, but you have numpy 1.19.5 which is incompatible.
pdpbox 0.2.1 requires matplotlib==3.1.1, but you have matplotlib 3.4.2 which is incompatible.
hypertools 0.7.0 requires scikit-learn!=0.22,<0.24,>=0.19.1, but you have scikit-learn 0.24.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from sklearn.linear_model import LogisticRegression,SGDClassifier
# #training the model
# lr=LogisticRegression(solver='liblinear',penalty='l2',max_iter=500,C=1,random_state=42)
# #Fitting the model for Bag of words
# lr_bow=lr.fit(train_x,train_y)
# print(lr_bow)

In [ ]:
#training the linear svm
svm=SGDClassifier(loss='hinge',max_iter=500,random_state=42)
#fitting the svm for bag of words
svm_bow=svm.fit(train_x,train_y)
print(svm_bow)
# #fitting the svm for tfidf features
# svm_tfidf=svm.fit(tv_train_reviews,train_sentiments)
# print(svm_tfidf)

SGDClassifier(max_iter=500, random_state=42)


In [ ]:
svm_bow_predict=svm.predict(test_x)
print(svm_bow_predict)
# #Predicting the model for tfidf features
# svm_tfidf_predict=svm.predict(tv_test_reviews)
# print(svm_tfidf_predict)

[2. 2. 2. ... 2. 2. 0.]


In [ ]:
svm_bow_score=accuracy_score(test_y,svm_bow_predict)
print("svm_bow_score :",svm_bow_score)

svm_bow_score : 0.36221390439958273


In [ ]:
from sklearn.svm import LinearSVC
classifier = LinearSVC(max_iter=10000, dual=True)
classifier.fit(train_x, train_y)
print(f"Accuracy: {classifier.score(test_x, test_y) * 100:.3f}%", )

Accuracy: 41.425%


/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  class_weight_, max_iter, rnd.randint(np.iinfo('i').max),
